## Блок 1. HashingTF и IDF
1) Подготовить фичи комментариев с помощью HashingTF и IDF

2) Обучить линейные классификаторы и сравнить метрики качествамоделей

3) Сделать выводы о влиянии параметра numFeatures в HashingTF накачество метрик



In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, Word2Vec
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import matplotlib as mpl

In [2]:
spark = SparkSession \
    .builder \
    .appName('Toxic comment') \
    .getOrCreate()

22/12/08 12:02:47 WARN Utils: Your hostname, MacBook-Pro-Faina.local resolves to a loopback address: 127.0.0.1; using 192.168.0.54 instead (on interface en0)
22/12/08 12:02:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 12:02:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train = (spark.read
      .option('header','True')
      .option('quote', '"')
      .option('escape', '"')
      .option('multiline', 'True')
      .csv('train.csv'))

test = (spark.read
      .option('header','True')
      .option('quote', '"')
      .option('escape', '"')
      .option('multiline', 'True')
      .csv('test.csv'))

test_labels = (spark.read
      .option('header','True')
      .option('quote', '"')
      .option('escape', '"')
      .option('multiline', 'True')
      .csv('test_labels.csv'))

test = test.join(test_labels, test.id == test_labels.id, "inner")


In [4]:
train.show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|Explanation\nWhy ...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|"\nMore\nI can't ...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
|00025465d4725e87|"\n\nCongratulati...|    0|           0|      0|     0|     0|            0|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|
|00031b1e95af7921|Your vandalism to...|    0|     

In [6]:
# Переведем все столбцы к числовому значению
train = train.withColumn("toxic", train["toxic"].cast(IntegerType()))
train = train.withColumn("severe_toxic", train["severe_toxic"].cast(IntegerType()))
train = train.withColumn("obscene", train["obscene"].cast(IntegerType()))
train = train.withColumn("threat", train["threat"].cast(IntegerType()))
train = train.withColumn("insult", train["insult"].cast(IntegerType()))
train = train.withColumn("identity_hate", train["identity_hate"].cast(IntegerType()))

test = test.withColumn("toxic", test["toxic"].cast(IntegerType()))
test = test.withColumn("severe_toxic", test["severe_toxic"].cast(IntegerType()))
test = test.withColumn("obscene", test["obscene"].cast(IntegerType()))
test = test.withColumn("threat", test["threat"].cast(IntegerType()))
test = test.withColumn("insult", test["insult"].cast(IntegerType()))
test = test.withColumn("identity_hate", test["identity_hate"].cast(IntegerType()))

In [53]:
def pipeline(train, label, gridNumFeatures):
    tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
    hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="featuresHF")
    idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="featuresTFIDF")
    lr = LogisticRegression(featuresCol='featuresTFIDF',
                            labelCol=label,
                            maxIter=10)
    pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

    paramGrid = ParamGridBuilder() \
        .addGrid(hashingTF.numFeatures, gridNumFeatures) \
        .build()

    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=BinaryClassificationEvaluator().setLabelCol(label),
                              numFolds=3,
                              seed = 1234)

    # Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    return cvModel

labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for label in labels:
    cvModel = pipeline(train, label, [20])
    evaluator = BinaryClassificationEvaluator(labelCol=label)
    predictions = cvModel.transform(test)
    print(f"Crossval score for {label} label:  {cvModel.avgMetrics[0]}")
    print(f"Evaluation on test for {label} label: {evaluator.evaluate(predictions)}")


22/12/08 13:03:19 WARN CacheManager: Asked to cache already cached data.
22/12/08 13:03:19 WARN CacheManager: Asked to cache already cached data.


Crossval score for toxic label:  0.6957387855533179


Evaluation on test for toxic label: 0.6541356374715067


Crossval score for severe_toxic label:  0.7547631462267118


Evaluation on test for severe_toxic label: 0.6454056627179952


...

Evaluation on test for threat label: 0.6993590603042165


Crossval score for insult label:  0.7282395899685272


Evaluation on test for insult label: 0.6780969320941418


Crossval score for identity_hate label:  0.6658652272391457


Evaluation on test for identity_hate label: 0.6597944707665587


Скор для тестовой выборки больше всего для предсказания threat, хуже для severe_toxic (хоть и скор на кросс валидации здесь выше всего). Посмотрим на зависимость обучения от параметра NumFeatures для лейбла threat

In [54]:
gridNumFeatures = [10, 20, 30, 40, 50, 60, 70, 80]
cvModel = pipeline(train, 'threat', gridNumFeatures)
print(list(zip(gridNumFeatures, cvModel.avgMetrics)))

[(10, 0.6735425721573103), (20, 0.692845788869684), (30, 0.6766676325145138), (40, 0.7310339832212859), (50, 0.7381091070576412), (60, 0.7515054462074433), (70, 0.75769814460171), (80, 0.767470476667302)]


По ходу обучения видно, что для NumFeatures = 80, скор самый высокий. Но для каждого лейбла следует смотреть этот параметр отдельно, либо принять для всех пара

## Блок 2. Word2Vec

1) Подготовить фичи комментариев с помощью w2v

2) Обучить линейный классификатор и сравнить метрики качества спредыдущими подходами

In [60]:
def pipeline_w2v(train, label, gridVecSize):
    tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
    word2Vec = Word2Vec(seed=1234, inputCol=tokenizer.getOutputCol(), outputCol="featuresW2V")

    lr = LogisticRegression(featuresCol='featuresW2V',
                            labelCol=label,
                            maxIter=10)
    pipeline = Pipeline(stages=[tokenizer, word2Vec, lr])

    paramGrid = ParamGridBuilder() \
        .addGrid(word2Vec.vectorSize, gridVecSize) \
        .build()

    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=BinaryClassificationEvaluator().setLabelCol(label),
                              numFolds=3,
                              seed = 1234)

    # Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    return cvModel

In [62]:
labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for label in labels:
    cvModel = pipeline_w2v(train, label, [5])
    evaluator = BinaryClassificationEvaluator(labelCol=label)
    predictions = cvModel.transform(test)
    print(f"Crossval score for {label} label:  {cvModel.avgMetrics[0]}")
    print(f"Evaluation on test for {label} label: {evaluator.evaluate(predictions)}")

Crossval score for toxic label:  0.8909961345541989


Evaluation on test for toxic label: 0.8124439339797731


Crossval score for severe_toxic label:  0.9580426705339712


Evaluation on test for severe_toxic label: 0.9253557715862388


Crossval score for obscene label:  0.9195509046376463


...

Evaluation on test for threat label: 0.8501881893341252


Crossval score for insult label:  0.9134482351043357


Evaluation on test for insult label: 0.8408644995902042


Crossval score for identity_hate label:  0.900358957971887


Evaluation on test for identity_hate label: 0.8736714412891624


Качество моделей с использованием word2vec намного выше для всех лейблов. Посмотрим значения скора для разного размера VecSize по лейблу severe_toxic.

In [63]:
gridVecSize = [5, 10, 15, 20, 25, 30]
cvModel = pipeline_w2v(train, 'severe_toxic', gridVecSize)
print(list(zip(gridVecSize, cvModel.avgMetrics)))

[(5, 0.9580426705339712), (10, 0.9646409212514432), (15, 0.9664250465869846), (20, 0.9698464331270799), (25, 0.969739924440807), (30, 0.9698489497127554)]


С увеличением длины векторного представления качество модели растет.